<a href="https://colab.research.google.com/github/aim56009/AI-MA_project/blob/main/Masterproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle

from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import glob
import os

In [2]:
!git clone https://github.com/aim56009/AI-MA_project.git

Cloning into 'AI-MA_project'...
remote: Enumerating objects: 1873, done.
remote: Counting objects: 100% (1873/1873), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 1873 (delta 1794), reused 1794 (delta 1755), pack-reused 0
Receiving objects: 100% (1873/1873), 3.92 MiB | 9.83 MiB/s, done.
Resolving deltas: 100% (1794/1794), done.
Checking out files: 100% (1845/1845), done.


# Dataloader

Maybe later: 

in dataloader:  pad_colate ----> consider only unpaded data for loss in training

 pad to longest seq in batch ..- better sort after len before loading batches



In [3]:
batch_size = 1      # needs pading for > 1
PATH_TO_DATA = "AI-MA_project/pianoroll"
workers = 0

In [4]:
class MusicDataset(Dataset):

    def __init__(self, data_dir, transforms=None):
        self.transforms = transforms
        self.data_dir = data_dir

        labels = ["voice_0", "voice_1", "voice_2", "voice_3", "voice_all"]
        self.labels = labels
        self.pr_dict = {}
                                                                                # load all files like below  (maybe do dicts instead of 1 big array)
                                                                                # make list of dict.  -voice_0 ..voice_all, length (-6 in total)
        len_list = []

        for iLabel in range(len(labels)):
            
            if iLabel == 4:   
                
                voice_files = []
                file_names = sorted(glob.glob(os.path.join(PATH_TO_DATA, self.labels[iLabel], "*.pkl")))   
                for name in file_names:
                    with open(name ,'rb') as f: 
                        loaded_obj = pickle.load(f)  
                        #loaded_obj = loaded_obj.T
                        voice_files.append(loaded_obj) 
                        len_list.append(len(loaded_obj.T))                             # for later: sort it after len(rows) when loading batches - to not have batches with len 1 and len 100 and then need to pad 99 zeros
                self.pr_dict[self.labels[iLabel]] = voice_files 
                self.pr_dict["length"] = len_list
            
            else:
                voice_files = []
                file_names = sorted(glob.glob(os.path.join(PATH_TO_DATA, self.labels[iLabel], "*.pkl")))   
                for name in file_names:
                    with open(name ,'rb') as f: 
                        loaded_obj = pickle.load(f)     
                        voice_files.append(loaded_obj)                              # for later: sort it after len(rows) when loading batches - to not have batches with len 1 and len 100 and then need to pad 99 zeros
                self.pr_dict[self.labels[iLabel]] = voice_files 
                                        

    def __len__(self):
      
        return len(self.pr_dict[self.labels[0]])

    def __getitem__(self, idx):                                                 ###maybe later revert sorted idx to file name idx

        out_list = []
        for key, value in self.pr_dict.items():
            out_list.append(self.pr_dict[key][idx])                  ##maybe shorten not saving it in list first
        v0 = torch.tensor(out_list[0].T)
        v1 = torch.tensor(out_list[1].T)
        v2 = torch.tensor(out_list[2].T)
        v3 = torch.tensor(out_list[3].T)
        v_all = torch.tensor(out_list[4].T) 

        length = self.pr_dict["length"]

        # v0.shape =torch.Size([816, 128])  , when doing v0, v1, v2, v3, v_all  = real , print(v0.shape) it is torch.Size([1, 756, 128])  - normal?

        return (v0, v1, v2, v3, v_all, length)

In [5]:
dataset = MusicDataset(PATH_TO_DATA)
loader = torch.utils.data.DataLoader(dataset,batch_size=batch_size, shuffle=False, num_workers=workers, drop_last=True)

### just for testing 

In [6]:
for batch_idx, real in enumerate(loader):
  v0, v1, v2, v3, v_all, length  = real
print(v0.shape)
  #print(v0.shape, v1.shape, v2.shape, v3.shape, v_all.shape)

torch.Size([1, 576, 128])


# Model - RNN

In [7]:
#https://github.com/fosfrancesco/pkspell/blob/main/src/models/models.py

In [8]:
class MusicRNN(nn.Module):

    def __init__(self, output_dim=128, hidden_dim=300, rnn_depth=1, cell_type="GRU"):                  #need to know input/output dim first
        super(MusicRNN, self).__init__()

        self.n_out = output_dim

        input_dim = output_dim 
        
        
        rnn_cell = nn.GRU

        self.rnn = rnn_cell(input_size=input_dim, hidden_size=hidden_dim, num_layers=rnn_depth, batch_first=True)


        # Output layers. The input will be two times
        self.top_layer_voice_0 = nn.Linear(hidden_dim, self.n_out)
        self.top_layer_voice_1 = nn.Linear(hidden_dim, self.n_out)
        self.top_layer_voice_2 = nn.Linear(hidden_dim, self.n_out)
        self.top_layer_voice_3 = nn.Linear(hidden_dim, self.n_out)

        # Loss function that we will use during training.
        self.loss = nn.CrossEntropyLoss(reduction="mean")


    def compute_outputs(self, sentences, sentences_len):

        # pack padded seq
        rnn_out ,_= self.rnn(sentences)
        # pad packed seq

        out_0 = self.top_layer_voice_0(rnn_out)
        out_1 = self.top_layer_voice_1(rnn_out)
        out_2 = self.top_layer_voice_2(rnn_out)
        out_3 = self.top_layer_voice_3(rnn_out)
        
        print("out_0.shape:",out_0.shape)
        print("out_1.shape:",out_1.shape)
        print("out_2.shape:",out_2.shape)
        print("out_3.shape:",out_3.shape)

        return out_0, out_1, out_2, out_3

    def forward(self, sentences, v0, v1, v2, v3, sentences_len):                # maybe change sent -> v_all

        # Compute the outputs. The shape is (max_len, n_sentences, n_labels).
        scores_0, scores_1, scores_2, scores_3 = self.compute_outputs(sentences, sentences_len)

        # Flatten the outputs and the labels, to compute the loss.
        # The input to this loss needs to be one 2-dimensional and one 1-dimensional tensor.
        score_0  = scores_0.view(-1, self.n_out)
        score_1  = scores_1.view(-1, self.n_out)
        score_2  = scores_2.view(-1, self.n_out)
        score_3  = scores_3.view(-1, self.n_out)

        # dont take padded len -> loss(score_0[:sentence_len], v0)      -> therefore store real len of voices  - before view


        v0 = v0.squeeze()
        v1 = v1.squeeze()
        v2 = v2.squeeze()
        v3 = v3.squeeze()

 
        
        loss = self.loss(score_0, v0) + self.loss(score_1, v1) + self.loss(score_2, v2) + self.loss(score_3, v3)
        
        return loss

    def predict(self, sentences, sentences_len):

        # Compute the outputs from the linear units.
        scores_0, scores_1, scores_2, scores_3 = self.compute_outputs(sentences, sentences_len)

        # Select the top-scoring labels. The shape is now (max_len, n_sentences).
        predicted_0 = scores_0.argmax(dim=2)
        predicted_1 = scores_1.argmax(dim=2)
        predicted_2 = scores_2.argmax(dim=2)
        predicted_3 = scores_3.argmax(dim=2)       # prbl distr over 0-not there, 1-there   - argmax -> take higher probl of those 2
        
        ##  predicted_X should be enough for BS = 1;  

        #print("simple pred:",predicted_0)
        #print("indexing pred:",[predicted_0[: int(l), i].cpu().numpy() for i, l in enumerate(sentences_len)])
        #print("test:",[predicted_0[: int(l), i].cpu().numpy() for i, l in enumerate(sentences_len)])
        
        # ([predicted_0[: int(l), i].cpu().numpy() for i, l in enumerate(sentences_len)],
        #        [predicted_1[: int(l), i].cpu().numpy() for i, l in enumerate(sentences_len)],
        #        [predicted_2[: int(l), i].cpu().numpy() for i, l in enumerate(sentences_len)],
        #        [predicted_3[: int(l), i].cpu().numpy() for i, l in enumerate(sentences_len)])
        return np.squeeze(predicted_0.cpu().numpy()), np.squeeze(predicted_1.cpu().numpy()), np.squeeze(predicted_2.cpu().numpy()), np.squeeze(predicted_3.cpu().numpy())

In [9]:
#start_experiment(epochs, lr, hidden_dim, bs, momentum, rnn_depth, device, cell_type, decay, learn_all)

# Train

In [10]:
import random
import click
import sklearn
import sklearn.model_selection
from collections import defaultdict
from sklearn.metrics import accuracy_score
from pathlib import Path
import sys

In [11]:
from torch import optim
from torch.optim import lr_scheduler

def train(epochs, lr, hidden_dim, momentum, rnn_depth, device, rnn_cell, weight_decay, train_dataloader, val_dataloader=None):
    
    output_dim = 128                                                            ### think about how to get the correct input=output size
    model = MusicRNN(output_dim, hidden_dim, rnn_depth, cell_type)              
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = lr_scheduler.MultiStepLR(optimizer, [epochs // 2], gamma=0.1, verbose=True)

    history = training_loop(model, optimizer, train_dataloader, epochs=epochs, val_dataloader=val_dataloader, device=device, scheduler=scheduler)

    return model, history

In [12]:
from sklearn.preprocessing import OneHotEncoder

In [54]:
def training_loop(model,optimizer, train_dataloader, epochs=50, val_dataloader=None, device=None, scheduler=None,):
    if device is None:
        device = (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
        print(f"Training on device: {device}")

    # Move model to GPU if needed
    model = model.to(device)
    history = defaultdict(list)

    for i_epoch in range(1, epochs + 1):
        loss_sum = 0
        accuracy_v0_sum = 0
        accuracy_v1_sum = 0
        accuracy_v2_sum = 0
        accuracy_v3_sum = 0
        accuracy_v_all_sum = 0
        model.train()
        

        for idx, (v0, v1, v2, v3, v_all, lens) in enumerate(train_dataloader):  

            v0, v1, v2, v3, v_all = (v0.to(device).float(), v1.to(device).float(), v2.to(device).float(), v3.to(device).float(), v_all.to(device).float())
            

            optimizer.zero_grad()
            loss = model.forward(v_all, v0, v1, v2, v3, lens)

            loss.backward()
            optimizer.step()
            loss_sum += loss.item()

            with torch.no_grad():
                pred_v0, pred_v1, pred_v2, pred_v3 = model.predict(v_all, lens)
                

                    # compute the accuracy without considering the padding
                print("v0 shape",v0.shape)
                print("pred shape",pred_v0.shape)
                print("v0 shape after trafo", np.squeeze(v0).argmax(dim=1).cpu().shape)
                print("pred shape after trafo",torch.nn.functional.one_hot(torch.tensor(pred_v0),num_classes=v0.shape[2]).shape)
                
                print("michi v0 shape after trafo", np.squeeze(v0).cpu().shape)
                print("michi pred shape after trafo",pred_v0.shape)


                acc_v0 = accuracy_score(torch.nn.functional.one_hot(torch.tensor(pred_v0),num_classes=v0.shape[2]), np.squeeze(v0).argmax(dim=1).cpu())
                
                ### make pred_v0 3 dim (one_hot) and make v0 1 dim - drop batch_dim (by queezeing) and then argmax -> make it 1D

                ## now I am thinking target/pred should have dim=N and Input = (N,C)
                acc_v1 = accuracy_score(torch.nn.functional.one_hot(torch.tensor(pred_v1),num_classes=v0.shape[2]), np.squeeze(v1).argmax(dim=1).cpu())
                acc_v2 = accuracy_score(torch.nn.functional.one_hot(torch.tensor(l),num_classes=v0.shape[2]), v2.cpu())
                acc_v3 = accuracy_score(torch.nn.functional.one_hot(torch.tensor(m),num_classes=v0.shape[2]), v3.cpu())
                    # normalize according to the number of sequences in the batch
                accuracy_v0_sum += acc_v0 / len(lens)
                accuracy_v1_sum += acc_v1 / len(lens)
                accuracy_v2_sum += acc_v2 / len(lens)
                accuracy_v3_sum += acc_v3 / len(lens)

        train_loss = loss_sum / len(train_dataloader)
        # normalize according to the number of batches
        train_accuracy_v0 = accuracy_v0_sum / len(train_dataloader)
        train_accuracy_v1 = accuracy_v1_sum / len(train_dataloader)
        train_accuracy_v2 = accuracy_v2_sum / len(train_dataloader)
        train_accuracy_v3 = accuracy_v3_sum / len(train_dataloader)

        history["train_loss"].append(train_loss)
        history["train_accuracy_v0"].append(train_accuracy_v0)
        history["train_accuracy_v1"].append(train_accuracy_v1)
        history["train_accuracy_v2"].append(train_accuracy_v2)
        history["train_accuracy_v3"].append(train_accuracy_v3)
        
        print("Train Loss: {}, Train Accuracy (Pitch): {}, Train Accuracy (KS): {}".format(
                train_loss, train_accuracy_v0, train_accuracy_v1, train_accuracy_v2, train_accuracy_v3))

        if val_dataloader is not None:
            # Evaluate on the validation set
            model.eval()
            accuracy_v0_sum = 0
            accuracy_v1_sum = 0
            accuracy_v2_sum = 0
            accuracy_v3_sum = 0

            with torch.no_grad():
                for v0, v1, v2, v3, v_all, lens in val_dataloader:
                    # Predict the model's output on a batch
                    predicted_v0, predicted_v1, predicted_v2, predicted_v3 = model.predict(v_all.to(device), lens)
                    # Update the lists that will be used to compute the accuracy
                    for i, (p, k, m ,l) in enumerate(zip(predicted_v0, predicted_v1, predicted_v2, predicted_v3)):
                        
                        # compute the accuracy without considering the padding
                        acc_v0 = accuracy_score(torch.nn.functional.one_hot(torch.tensor(p),num_classes=v0.shape[2]), v0[:, i][: len(p)].cpu())
                        acc_v1 = accuracy_score(torch.nn.functional.one_hot(torch.tensor(k),num_classes=v0.shape[2]), v1[:, i][: len(k)].cpu())
                        acc_v2 = accuracy_score(torch.nn.functional.one_hot(torch.tensor(m),num_classes=v0.shape[2]), v2[:, i][: len(m)].cpu())
                        acc_v3 = accuracy_score(torch.nn.functional.one_hot(torch.tensor(l),num_classes=v0.shape[2]), v3[:, i][: len(l)].cpu())
                        
                        # normalize according to the number of sequences in the batch
                        accuracy_v0_sum += acc_v0 / len(lens)
                        accuracy_v1_sum += acc_v1 / len(lens)
                        accuracy_v2_sum += acc_v2 / len(lens)
                        accuracy_v3_sum += acc_v3 / len(lens)

                # normalize according to the number of batches
                val_accuracy_v0 = accuracy_v0_sum / len(val_dataloader)
                val_accuracy_v1 = accuracy_v1_sum / len(val_dataloader)
                val_accuracy_v2 = accuracy_v2_sum / len(val_dataloader)
                val_accuracy_v3 = accuracy_v3_sum / len(val_dataloader)

            history["val_accuracy_v0"].append(val_accuracy_v0)
            history["val_accuracy_v1"].append(val_accuracy_v1)
            history["val_accuracy_v2"].append(val_accuracy_v2)
            history["val_accuracy_v3"].append(val_accuracy_v3)
            
            print("Validation Accuracy (Pitch): {}, Validation Accuracy (KS): {}".format(val_accuracy_v0, val_accuracy_v1, val_accuracy_v2, val_accuracy_v3))

        if scheduler is not None:
            scheduler.step()

        # save the model
        torch.save(model, Path("./AI-MA_project/model_temp_epoch{}.pkl".format(i_epoch)))

    return history

In [55]:
start_experiment(epochs, lr, hidden_dim, bs, momentum, rnn_depth, device, cell_type, decay, learn_all)

#print(next(model.parameters()).is_cuda)

Learning from full dataset
Adjusting learning rate of group 0 to 1.0000e-03.
Training on device: cuda
out_0.shape: torch.Size([1, 756, 128])
out_1.shape: torch.Size([1, 756, 128])
out_2.shape: torch.Size([1, 756, 128])
out_3.shape: torch.Size([1, 756, 128])
out_0.shape: torch.Size([1, 756, 128])
out_1.shape: torch.Size([1, 756, 128])
out_2.shape: torch.Size([1, 756, 128])
out_3.shape: torch.Size([1, 756, 128])
v0 shape torch.Size([1, 756, 128])
pred shape (756,)
v0 shape after trafo torch.Size([756])
pred shape after trafo torch.Size([756, 128])
michi v0 shape after trafo torch.Size([756, 128])
michi pred shape after trafo (756,)


ValueError: ignored

In [27]:
from functools import partial

def start_experiment(epochs, lr, hidden_dim, bs, momentum, rnn_depth, device, cell, decay, learn_all):
    

    trainer = partial(train, epochs, lr, hidden_dim, momentum, rnn_depth, device, cell, decay)

    if learn_all:
        print("Learning from full dataset")
        train_dataset = MusicDataset(PATH_TO_DATA)
        train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size, shuffle=False, num_workers=workers, drop_last=True)
                
        _, history = trainer(train_dataloader)
    
    else:
        paths = np.array(paths)
        # Divide train and validation set
        path_train, path_validation = sklearn.model_selection.train_test_split(PATH_TO_DATA, test_size=0.15, random_state=seed,)

        print("Train and validation lenghts: ", len(path_train), len(path_validation))
        train_dataset = MusicDataset(path_train)
        validation_dataset = MusicDataset(path_validation)

        _, history = trainer(
            train_dataloader(train_dataset), val_dataloader(validation_dataset))

# Hyperparameter choice

In [28]:
model = MusicRNN
epochs = 1
lr = 0.01
momentum = 0.9
decay = 1e-4
hidden_dim = 300
bs = 1
rnn_depth = 1
device = None #torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")            ---- if None:  choses device automatically
cell_type = "GRU"
optimizer = "Adam"
learn_all = "True"      # change later to false to see if learning works

In [17]:
start_experiment(epochs, lr, hidden_dim, bs, momentum, rnn_depth, device, cell_type, decay, learn_all)

Learning from full dataset
Adjusting learning rate of group 0 to 1.0000e-03.
Training on device: cuda
out_0.shape: torch.Size([1, 756, 128])
out_1.shape: torch.Size([1, 756, 128])
out_2.shape: torch.Size([1, 756, 128])
out_3.shape: torch.Size([1, 756, 128])
out_0.shape: torch.Size([1, 756, 128])
out_1.shape: torch.Size([1, 756, 128])
out_2.shape: torch.Size([1, 756, 128])
out_3.shape: torch.Size([1, 756, 128])


TypeError: ignored